# Задача 3

## Постановка задачи
---
Склад пункта реализации станков имеет вместимость 20 единиц. Пополнение склада возможно только первого числа каждого месяца. Станки привозят автотранспортом (1 рейс), причем
стоимость рейса складывается из постоянных затрат 50 денежных единиц (ДЕ) и затрат на доставку каждого станка 10 ДЕ. За один рейс (и, следовательно, в месяц) может быть доставлено
не более 5 станков.
Затраты на хранение станка в течение месяца составляют 10 ДЕ.
Ожидаемый спрос на станки приведен в табл. 1. Отсутствие требуемого количества станков
на складе недопустимо.
В начальный момент на складе находится 4 станков.
1. определить план заказов, минимизирующий стоимость;
1. определить границы изменения стоимости хранения станка, в которых найденный план является оптимальным;
1. определить границы изменения постоянных затрат на совершение рейса, в которых найденный план является оптимальным;
1. определить план заказов, минимизирующий стоимость, при условии, что к концу периода на складе должно остаться 3 станков.

Таблица 1: Ожидаемый спрос на станки по месяцам

|  | 1 | 2 | 3 | 4 | 5 | 6 |
|---|:---:|:---:|:---:|:---:|:---:|:---:|
| Спрос, шт. | 0 | 7 | 6 | 3 | 0 | 2 |

## Решение задачи
Данная задача - задача на динамическое программирование. Решение с подробными комментариями на языке Python представлено ниже. В решении были применены рекурсия и мемоизация. Для сокрытия деталей реализации было решено использовать класс.

In [1]:
class Solver:
    def __init__(self):
        self.__warehouse_capacity = 20 # вместимость склада 0..20 - состояния
        self.__start_remainder = 4 # изначальный остаток на складе
        self.__unit_storage_cost = 10 # стоимость хранения единицы
        self.__delivery_limit = 5 # ограничение на количество 0..5 управления
        self.__delivery_cost = 50 # постоянные затраты на совершение рейса
        self.__unit_delivery_cost = 10 # доставка единицы
        self.__end_remainder = 0 # конечный остаток на складе
        
        self.__expected_demand = [0, 7, 6, 3, 0, 2] # ожидаемый спрос по месяцам stages

        self.__cache = [{} for _ in self.__expected_demand]
        
    # Установить размер стоимости хранения станка
    def SetUnitStorageCost(self, cost):
        self.__unit_storage_cost = cost
        return self
        
    # Установить размер постоянных затрат на доставку
    def SetDeliveryCost(self, cost):
        self.__delivery_cost = cost
        return self
        
    # Установить конечный остаток (для п.4)
    def SetEndRemainder(self, count):
        self.__end_remainder = count
        return self
        
    # Расчет стоимости доставки
    def __CalcDeliveryCost(self, count):
        return 0 if count == 0 else self.__delivery_cost + self.__unit_delivery_cost * count
    
    # Рекурсивная функция
    def __W(self, month, remainder):
        if month >= len(self.__expected_demand):
            if remainder == self.__end_remainder:
                return (0, "")
            else:
                return None
        
        if remainder in self.__cache[month]:
            return self.__cache[month][remainder];
    
        best = None
    
        cur_storage_cost = remainder * self.__unit_storage_cost
        restrict = lambda n: self.__expected_demand[month] <= remainder + n <= self.__warehouse_capacity
        for n in filter(restrict, range(self.__delivery_limit + 1)):
            wi = self.__W(month + 1, remainder + n - self.__expected_demand[month])
            if wi is not None:
                cur_cost = cur_storage_cost + self.__CalcDeliveryCost(n) + wi[0]
                if best is None or cur_cost < best[0]:
                    best = cur_cost, str(n) + wi[1]
                    
        self.__cache[month][remainder] = best      
        
        return best
    
    def Solve(self):
        return self.__W(0, self.__start_remainder)

In [2]:
### Проверка скорости с мемоизацией
import time
start = time.time()
Solver().Solve()
print('{} sec'.format(time.time() - start))

0.0 sec


### Определить план заказов, минимизирующий стоимость


In [3]:
cost, seq = Solver().Solve()
cost, seq

(420, '045500')

Лучший вариант работы склада обойдется в __420 у.е.__

Таблица 2: Ожидаемый спрос и план заказов  
|  | 1 | 2 | 3 | 4 | 5 | 6 |
|---|:---:|:---:|:---:|:---:|:---:|:---:|
| Спрос, шт. | 0 | 7 | 6 | 3 | 0 | 2 |
| Заказ в текущем месяце, шт. | 0 | 4 | 5 | 5 | 0 | 0 |

### Определить границы изменения стоимости хранения станка, в которых найденный план является оптимальным

In [4]:
r = list(filter(lambda i: seq == Solver().SetUnitStorageCost(i).Solve()[1], range(300)))
print("bounds: ", min(r), max(r))

bounds:  0 12


План является оптимальным, если стоимость хранения станка не превышает __12у.е.__

### Определить границы изменения постоянных затрат на совершение рейса, в которых найденный план является оптимальным

In [5]:
r = list(filter(lambda i: seq == Solver().SetDeliveryCost(i).Solve()[1], range(3000)))
print("bounds: ", min(r), max(r))

bounds:  41 2999


План является оптимальным, если стоимость постоянных затрат на совершение рейса не ниже __41у.е.__

### Определить план заказов, минимизирующий стоимость, при условии, что к концу периода на складе должно остаться 3 станков

In [6]:
Solver().SetEndRemainder(3).Solve()

(460, '045305')

Лучший вариант работы склада при условии остатка 3х станков обойдется в __460 у.е.__

Таблица 3: Ожидаемый спрос и план заказов при условии остатка 3х станков
|  | 1 | 2 | 3 | 4 | 5 | 6 |
|---|:---:|:---:|:---:|:---:|:---:|:---:|
| Спрос, шт. | 0 | 7 | 6 | 3 | 0 | 2 |
| Заказ в текущем месяце, шт. | 0 | 4 | 5 | 3 | 0 | 5 |